Download and Unpack the Urban Sound 8K Dataset

In [ ]:
!wget 'https://zenodo.org/record/1203745/files/UrbanSound8K.tar.gz'
!tar xvzf './UrbanSound8K.tar.gz'

Streaming output truncated to the last 5000 lines.
UrbanSound8K/audio/fold4/17480-2-0-6.wav
UrbanSound8K/audio/fold4/17480-2-0-9.wav
UrbanSound8K/audio/fold4/175904-2-0-11.wav
UrbanSound8K/audio/fold4/175904-2-0-24.wav
UrbanSound8K/audio/fold4/176003-1-0-0.wav
UrbanSound8K/audio/fold4/176638-5-0-0.wav
UrbanSound8K/audio/fold4/177756-2-0-10.wav
UrbanSound8K/audio/fold4/177756-2-0-4.wav
UrbanSound8K/audio/fold4/177756-2-0-5.wav
UrbanSound8K/audio/fold4/177756-2-0-7.wav
UrbanSound8K/audio/fold4/179862-1-0-0.wav
UrbanSound8K/audio/fold4/180977-3-1-1.wav
UrbanSound8K/audio/fold4/180977-3-1-5.wav
UrbanSound8K/audio/fold4/183989-3-1-21.wav
UrbanSound8K/audio/fold4/183989-3-1-23.wav
UrbanSound8K/audio/fold4/185709-0-0-0.wav
UrbanSound8K/audio/fold4/185709-0-0-1.wav
UrbanSound8K/audio/fold4/185709-0-0-6.wav
UrbanSound8K/audio/fold4/185709-0-0-7.wav
UrbanSound8K/audio/fold4/185909-2-0-102.wav
UrbanSound8K/audio/fold4/185909-2-0-13.wav
UrbanSound8K/audio/fold4/185909-2-0-17.wav
UrbanSound8K/audio

Data Preprocessing

In [ ]:
import librosa
import pandas as pd
import os
fulldatasetpath = './UrbanSound8K/audio/'
metadata = pd.read_csv('./UrbanSound8K/metadata/UrbanSound8K.csv')

Extract Features Using MFCC

In [ ]:
def feature_extractor(filename):
  audio, sample_rate = librosa.load(filename)
  mfcc = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
  mfcc_scaled = np.mean(mfcc.T, axis=0)
  return mfcc_scaled

In [ ]:
import numpy as np
from tqdm import tqdm
features = []
for ind, row in tqdm(metadata.iterrows()):
  filename = os.path.join(os.path.abspath(fulldatasetpath), 'fold'+str(row["fold"])+'/',str(row["slice_file_name"]))
  class_label = row["class"]
  data = feature_extractor(filename)
  features.append([data, class_label])

3555it [14:12,  4.28it/s]/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=2048 is too small for input signal of length=1323
  n_fft, y.shape[-1]
8325it [32:31,  5.99it/s]/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=2048 is too small for input signal of length=1103
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=2048 is too small for input signal of length=1523
  n_fft, y.shape[-1]
8732it [33:59,  4.28it/s]


Convert into DataFrame

In [ ]:
featuresdf = pd.DataFrame(features, columns=['feature','class_label'])
X = np.array(featuresdf['feature'].to_list())
y = np.array(featuresdf['class_label'].to_list())

Label Encoding

In [ ]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = to_categorical(le.fit_transform(y))

Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Model Creation

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score
from keras.layers import Convolution2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, Dropout, Activation, Flatten
# num_rows = 40
# num_columns = 174
# num_channels = 1
# X_train = X_train.reshape(X_train.shape[0], num_rows, num_columns, num_channels)
# X_test = X_test.reshape(X_test.shape[0], num_rows, num_columns, num_channels)

num_labels = y.shape[1]
# filter_size = 2

model = Sequential()
# model.add(Conv2D(filters=16, kernel_size=2, input_shape=(num_rows, num_columns, num_channels), activation='relu'))
# model.add(Dense(16, input_shape=(40,)))
# model.add(Conv2D(filters=16, kernel_size=2, activation="relu"))
# model.add(MaxPooling2D(pool_size=2))
# model.add(Dropout(0.2))

# model.add(Conv2D(filters=32, kernel_size=2, activation='relu'))
# model.add(MaxPooling2D(pool_size=2))
# model.add(Dropout(0.2))

# model.add(Conv2D(filters=64, kernel_size=2, activation='relu'))
# model.add(MaxPooling2D(pool_size=2))
# model.add(Dropout(0.2))

# model.add(Conv2D(filters=128, kernel_size=2, activation='relu'))
# model.add(MaxPooling2D(pool_size=2))
# model.add(Dropout(0.2))
# model.add(GlobalAveragePooling2D())

# model.add(Dense(num_labels, activation='softmax'))
model.add(Dense(100, input_shape=(40, )))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(200))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(num_labels))
model.add(Activation("softmax"))
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 100)               4100      
_________________________________________________________________
activation_6 (Activation)    (None, 100)               0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 200)               20200     
_________________________________________________________________
activation_7 (Activation)    (None, 200)               0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 200)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 10)              

Training

In [ ]:
num_epochs = 72
num_batch_size = 256
from datetime import datetime
start = datetime.now()
model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_test, y_test), verbose=1)
duration = datetime.now() - start
print("Training completed in time: ", duration)

Epoch 1/72
28/28 [==============================] - 1s 17ms/step - loss: 38.5599 - accuracy: 0.1106 - val_loss: 4.1765 - val_accuracy: 0.2576
Epoch 2/72
28/28 [==============================] - 0s 7ms/step - loss: 13.6871 - accuracy: 0.1489 - val_loss: 1.9991 - val_accuracy: 0.3406
Epoch 3/72
28/28 [==============================] - 0s 7ms/step - loss: 6.8724 - accuracy: 0.1841 - val_loss: 1.9362 - val_accuracy: 0.3280
Epoch 4/72
28/28 [==============================] - 0s 12ms/step - loss: 4.2767 - accuracy: 0.2009 - val_loss: 2.0165 - val_accuracy: 0.2862
Epoch 5/72
28/28 [==============================] - 0s 7ms/step - loss: 3.3613 - accuracy: 0.2112 - val_loss: 2.0682 - val_accuracy: 0.2639
Epoch 6/72
28/28 [==============================] - 0s 7ms/step - loss: 2.8156 - accuracy: 0.2096 - val_loss: 2.0894 - val_accuracy: 0.2456
Epoch 7/72
28/28 [==============================] - 0s 7ms/step - loss: 2.4117 - accuracy: 0.2259 - val_loss: 2.0877 - val_accuracy: 0.2461
Epoch 8/72
28/28

Accuracy Score

In [ ]:
score = model.evaluate(X_train, y_train, verbose=0)
print("Training Accuracy: ", score[1])
score = model.evaluate(X_test, y_test, verbose=0)
print("Testing Accuracy: ", score[1])

Training Accuracy:  0.7620615363121033
Testing Accuracy:  0.7447052001953125


Predict a Sound File

In [ ]:
def print_prediction(file_name):
    prediction_feature = extract_features(file_name) 
    prediction_feature = prediction_feature.reshape(1, num_rows, num_columns, num_channels)
    predicted_vector = model.predict_classes(prediction_feature)
    predicted_class = le.inverse_transform(predicted_vector) 
    print("The predicted class is:", predicted_class[0], '\n') 
    predicted_proba_vector = model.predict_proba(prediction_feature) 
    predicted_proba = predicted_proba_vector[0]
    for i in range(len(predicted_proba)): 
        category = le.inverse_transform(np.array([i]))
        print(category[0], "\t\t : ", format(predicted_proba[i], '.32f'))